# Barcelona Bicycle Station Analysis and Prediction

## Objective
This notebook aims to analyze data from Barcelona's bicycle stations and predict the percentage of free docks at a given time based on historical data.

## Data Sources
We have two datasets:
1. Historical data - Contains time-series information about station status
2. Station data - Contains static information about each station

We'll merge these datasets to create a comprehensive view for our analysis and prediction model.

## 1. Import Libraries

In [1]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import os
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy import stats

# Date and time handling
from datetime import datetime, timedelta

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import lightgbm as lgb
import optuna

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


## 2. Load and Prepare Data

### 2.1 Download Historical Data

In [2]:
# import os
# i2m = list(zip(range(1,13), ['Gener','Febrer','Marc','Abril','Maig','Juny','Juliol','Agost','Setembre','Octubre','Novembre','Desembre']))
# for year in range(2024, 2019, -1):
#     for month, month_name in i2m:
#         if (month > 5) and (year>2023): continue
#         print(f'curl -L -o "{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z" "https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z"')

### 2.2 Load Station Data

In [3]:
def load_station_data():
    # Load station data or create sample data if file not found
    try:
        station_data = pd.read_csv('data/station_data/Informacio_Estacions_Bicing_2025.csv')
        print(f"Station data loaded successfully with {station_data.shape[0]} rows and {station_data.shape[1]} columns")
        return station_data
    except FileNotFoundError:
        print("Station data file not found. Creating sample data for demonstration...")
    

### 2.3 Convert Timestamps and Handle Missing Values Function

In [4]:
def prepare_historical_data(historical_data):
    #Drop not needed columns
    historical_data.drop(columns=['num_bikes_available_types.ebike',  'ttl',
       'num_bikes_available_types.mechanical', 'num_bikes_available', 'last_updated'], inplace=True)

    # Convert timestamp columns to datetime format
    try:
        historical_data['last_reported'] = pd.to_datetime(historical_data['last_reported'], unit='s')

    except Exception as e:
        print(f"Error converting timestamps: {e}")

    # Handle missing values in historical data
    numerical_cols = ['num_docks_available']
    for col in numerical_cols:
        if col in historical_data.columns and historical_data[col].isnull().sum() > 0:
            historical_data[col].fillna(historical_data[col].median(), inplace=True)
    return historical_data

def prepare_station_data(station_data):

    #Drop not needed columns
    station_data.drop(columns=['name', 'address',
       'cross_street', 'short_name', 'nearby_distance', '_ride_code_support', 
       'rental_uris', 'is_valet_station', 'physical_configuration', 'is_charging_station', 'post_code'], inplace=True)

    # Handle missing values in station data
    numerical_cols = ['lat', 'lon', 'altitude', 'capacity']
    for col in numerical_cols:
        if col in station_data.columns and station_data[col].isnull().sum() > 0:
            station_data[col].fillna(station_data[col].median(), inplace=True)
    return station_data    

### 2.4 Create Hourly Average Dataset Function

In [5]:
# Create a new dataset with hourly averages of available bikes per station and date
def create_hourly_avg_dataset(historical_data):

    # Extract date components
    historical_data['year'] = historical_data['last_reported'].dt.year
    historical_data['month'] = historical_data['last_reported'].dt.month
    historical_data['day'] = historical_data['last_reported'].dt.day
    historical_data['hour'] = historical_data['last_reported'].dt.hour
    historical_data['date'] = pd.to_datetime(historical_data[['year', 'month', 'day']])

    # Group by station_id, year, month, day, hour and calculate average of num_bikes_available
    hourly_avg = historical_data.groupby(['station_id', 'year', 'month', 'day', 'hour', 'date'])['num_docks_available'].mean().reset_index()

    return hourly_avg


### 2.5 Merge data Function

In [6]:
# Merge data with station data on station_id
def merge_data(data,station_data):
    merged_data = pd.merge(data, station_data, on='station_id', how='inner')
    return merged_data

### 2.6 Feature Engineering Function

In [7]:
def create_additional_features(data):
    # Add time-based features to historical data
    data['day_of_week'] = data['date'].dt.dayofweek
    data['is_weekend'] = data['day_of_week'].isin([5, 6]).astype(int)

    # Add distance from city center for each station
    # Barcelona city center coordinates (Plaça de Catalunya)
    barcelona_center_lat = 41.3874
    barcelona_center_lon = 2.1686

    if all(col in data.columns for col in ['lat', 'lon']):
        # Calculate Haversine distance (in kilometers)
        def haversine_distance(lat1, lon1, lat2, lon2):
            # Convert decimal degrees to radians
            lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

            # Haversine formula
            dlon = lon2 - lon1
            dlat = lat2 - lat1
            a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
            c = 2 * np.arcsin(np.sqrt(a))
            r = 6371  # Radius of Earth in kilometers
            return c * r

        data['distance_from_center'] = data.apply(
            lambda row: haversine_distance(row['lat'], row['lon'], barcelona_center_lat, barcelona_center_lon),
            axis=1
        ).round(3)
    
    return data

def create_free_docks_percentage(data):
    # Calculate the percentage of free docks (our target variable)
    data[f'free_docks_percentage'] = (data[f'num_docks_available'] / data['capacity']).round(2)
    return data

def drop_other_features(data):
    data.drop(columns=['num_docks_available', 'date'], inplace=True)
    return data

### 2.7 Create final dataset

In [8]:
station_data = load_station_data()
station_data = prepare_station_data(station_data)

csv_files = [f for f in os.listdir("./data") if f.endswith('.csv')]

for file in csv_files:
    print(f'File {file} started!')
    file_path = os.path.join("./data", file)
    data = pd.read_csv(file_path)

    data = prepare_historical_data(data)
    data = create_hourly_avg_dataset(data)
    data = merge_data(data, station_data)
    data = create_additional_features(data)
    data = create_free_docks_percentage(data)
    data = drop_other_features(data)

    write_header = not os.path.exists("./data/historical_data.csv")
    data.to_csv("./data/historical_data.csv", mode='a', index=False, header=write_header)
    print(f"File loaded successfully with {data.shape[0]} rows.")


Station data loaded successfully with 516 rows and 16 columns
File 2020_01_Gener_BicingNou_ESTACIONS.csv started!
File loaded successfully with 274832 rows.
File 2020_02_Febrer_BicingNou_ESTACIONS.csv started!
File loaded successfully with 284373 rows.
File 2020_03_Marc_BicingNou_ESTACIONS.csv started!
File loaded successfully with 162924 rows.
File 2020_04_Abril_BicingNou_ESTACIONS.csv started!
File loaded successfully with 101422 rows.
File 2020_05_Maig_BicingNou_ESTACIONS.csv started!
File loaded successfully with 308279 rows.
File 2020_06_Juny_BicingNou_ESTACIONS.csv started!
File loaded successfully with 339627 rows.
File 2020_07_Juliol_BicingNou_ESTACIONS.csv started!
File loaded successfully with 357907 rows.
File 2020_08_Agost_BicingNou_ESTACIONS.csv started!
File loaded successfully with 320242 rows.
File 2020_09_Setembre_BicingNou_ESTACIONS.csv started!
File loaded successfully with 350666 rows.
File 2020_10_Octubre_BicingNou_ESTACIONS.csv started!
File loaded successfully wi

## 3. Load complete dataset

In [9]:
# Load final dataset
try:
    complete_data = pd.read_csv('data/historical_data.csv')
    print(f"Final data loaded successfully with {complete_data.shape[0]} rows and {complete_data.shape[1]} columns")
except FileNotFoundError:
    print("Station data file not found. Creating sample data for demonstration...")

Final data loaded successfully with 18209862 rows and 13 columns


## 4. Explore data

In [10]:
# ========================
# Basic Info
# ========================
print("Shape:", complete_data.shape)
print(complete_data.info())

# ========================
# Missing Values Analysis
# ========================
missing_counts = complete_data.isnull().sum()
missing_percentage = (missing_counts / len(complete_data)) * 100

missing_df = pd.DataFrame({'Missing Values': missing_counts, 'Percentage': missing_percentage})
print(missing_df[missing_df['Missing Values'] > 0].sort_values(by='Percentage', ascending=False))

# Filter rows where lat or lon is missing
missing_location = complete_data[complete_data[['lat', 'lon']].isnull().any(axis=1)]

# Check which station_ids have missing location data
stations_with_missing_location = missing_location['station_id'].unique()

print(f"Total records with missing lat/lon: {missing_location.shape[0]}")
print(f"Station IDs with missing location data: {stations_with_missing_location}")



Shape: (18209862, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18209862 entries, 0 to 18209861
Data columns (total 13 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   station_id             float64
 1   year                   float64
 2   month                  float64
 3   day                    float64
 4   hour                   float64
 5   lat                    float64
 6   lon                    float64
 7   altitude               float64
 8   capacity               int64  
 9   day_of_week            int64  
 10  is_weekend             int64  
 11  distance_from_center   float64
 12  free_docks_percentage  float64
dtypes: float64(10), int64(3)
memory usage: 1.8 GB
None
Empty DataFrame
Columns: [Missing Values, Percentage]
Index: []
Total records with missing lat/lon: 0
Station IDs with missing location data: []


## 5. Create LAG Features for Model Training

In [11]:
df = complete_data.sort_values(by=['station_id', 'year', 'month', 'day', 'hour'])

# Create lag features
for lag in range(1, 5):
    complete_data[f'ctx-{lag}'] = complete_data.groupby('station_id')['free_docks_percentage'].shift(lag)

ctx_columns = ['ctx-1', 'ctx-2', 'ctx-3', 'ctx-4']

# Drop rows where any of these columns is NaN
complete_data = complete_data.dropna(subset=ctx_columns).reset_index(drop=True)


## 6. Split Data and Train Model

### 6.1 Prepare features and target

In [12]:
# Define the target
target = 'free_docks_percentage'

# Define the features
features = ['station_id', 'year', 'month', 'day', 'hour', 'lat', 'lon', 'altitude',
            'capacity', 'day_of_week', 'is_weekend', 'distance_from_center',
            'ctx-1', 'ctx-2', 'ctx-3', 'ctx-4']

X = complete_data[features]
y = complete_data[target]

### 6.2 Split the data

In [13]:
# First split: Train + Val and Test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Second split: Train and Val
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, shuffle=True)
# Result: 60% train, 20% val, 20% test

train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)

### 6.3 Train the model

In [14]:

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'feature_pre_filter': False,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }
    model = lgb.train(params, train_data, valid_sets=[val_data], callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(0)])
    preds = model.predict(X_val)
    rmse = mean_squared_error(y_val, preds)
    return rmse

# Run tuning
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Best parameters
best_params = study.best_trial.params
print("Best Hyperparameters:", best_params)

# ========================
# 4. Train final model
# ========================
final_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    **best_params
}

model = lgb.train(
    final_params,
    train_data,
    valid_sets=[val_data],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(50)]
)
model.save_model('lightgbm_free_docks_model.txt')

[I 2025-03-27 12:40:31,054] A new study created in memory with name: no-name-fa53e3f1-720a-4d53-810f-fb23a29d3714


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.102175


[I 2025-03-27 12:41:34,492] Trial 0 finished with value: 0.010439681553858795 and parameters: {'learning_rate': 0.048989029148989234, 'num_leaves': 60, 'max_depth': 6, 'min_data_in_leaf': 99, 'feature_fraction': 0.9805615770217293, 'bagging_fraction': 0.6178382985419547, 'bagging_freq': 7}. Best is trial 0 with value: 0.010439681553858795.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.0972207


[I 2025-03-27 12:43:16,933] Trial 1 finished with value: 0.009451865619860632 and parameters: {'learning_rate': 0.25228929520822735, 'num_leaves': 246, 'max_depth': 8, 'min_data_in_leaf': 29, 'feature_fraction': 0.8385602398609304, 'bagging_fraction': 0.9382375139085373, 'bagging_freq': 3}. Best is trial 1 with value: 0.009451865619860632.


...

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.103289


[I 2025-03-27 12:50:38,837] Trial 8 finished with value: 0.010668542743839849 and parameters: {'learning_rate': 0.02996591290441218, 'num_leaves': 167, 'max_depth': 8, 'min_data_in_leaf': 78, 'feature_fraction': 0.7944998054495347, 'bagging_fraction': 0.7546107134776684, 'bagging_freq': 6}. Best is trial 1 with value: 0.009451865619860632.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.0980032


[I 2025-03-27 12:51:38,166] Trial 9 finished with value: 0.009604620267305944 and parameters: {'learning_rate': 0.281216136367123, 'num_leaves': 121, 'max_depth': 11, 'min_data_in_leaf': 83, 'feature_fraction': 0.6287564718110673, 'bagging_fraction': 0.9046432642365693, 'bagging_freq': 1}. Best is trial 1 with value: 0.009451865619860632.


Best Hyperparameters: {'learning_rate': 0.25228929520822735, 'num_leaves': 246, 'max_depth': 8, 'min_data_in_leaf': 29, 'feature_fraction': 0.8385602398609304, 'bagging_fraction': 0.9382375139085373, 'bagging_freq': 3}
Training until validation scores don't improve for 50 rounds
[50]	valid_0's rmse: 0.0985685
[100]	valid_0's rmse: 0.0972207
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.0972207


## 7. Evaluate Model

In [15]:
# Define evaluation function
def evaluate_model(model, X, y, set_name):
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)

    print(f"{set_name} set metrics:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R²: {r2:.4f}")

    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2
    }

In [16]:
evaluate_model(model, X_train, y_train, "Train")
evaluate_model(model, X_val, y_val, "Validation")
evaluate_model(model, X_test, y_test, "Test")

Train set metrics:
  MSE: 0.0094
  RMSE: 0.0970
  MAE: 0.0626
  R²: 0.8768
Validation set metrics:
  MSE: 0.0095
  RMSE: 0.0972
  MAE: 0.0628
  R²: 0.8762
Test set metrics:
  MSE: 0.0095
  RMSE: 0.0973
  MAE: 0.0628
  R²: 0.8760


{'mse': 0.009471682580178659,
 'rmse': 0.09732256973682239,
 'mae': 0.06284088405545417,
 'r2': 0.8759560213790335}

## 8 Prepare Submission

### 8.1 Load Metadata Submission

In [17]:
def load_submission_metadata():
    # Load metadata
    try:
        submission_data = pd.read_csv('sample/metadata_sample_submission_2025.csv')
        print(f"Station data loaded successfully with {submission_data.shape[0]} rows and {submission_data.shape[1]} columns")
        return submission_data
    except FileNotFoundError:
        print("Station data file not found. Creating sample data for demonstration...")

submission_data = load_submission_metadata()

Station data loaded successfully with 401511 rows and 9 columns


### 8.2 Prepare Subsmission Data

In [18]:
station_data = load_station_data()
station_data = prepare_station_data(station_data)

submission_data = merge_data(submission_data, station_data)
submission_data['year'] = 2024
submission_data['date'] = pd.to_datetime(submission_data[['year', 'month', 'day']])
submission_data = create_additional_features(submission_data)
submission_data = submission_data.drop(columns=['index','date'])

submission_data = submission_data[X_train.columns]

# Check if the columns of prediction_data match the columns of X_train
columns_match = submission_data.columns.tolist() == X_train.columns.tolist()
# Print if the columns match
print("Columns match:", columns_match)

# If the columns don't match, print the differences
if not columns_match:
    print("Columns in prediction_data that are not in X_train:", set(submission_data.columns) - set(X_train.columns))
    print("Columns in X_train that are not in prediction_data:", set(X_train.columns) - set(submission_data.columns))


Station data loaded successfully with 516 rows and 16 columns
Columns match: True


### 8.3 Predict

In [19]:
submission_data.head()

,station_id,year,month,day,hour,lat,lon,altitude,capacity,day_of_week,is_weekend,distance_from_center,ctx-1,ctx-2,ctx-3,ctx-4
0,1,2024,6,1,3,41.397978,2.180107,16.0,46,5,1,1.518,0.311594,0.324275,0.378623,0.490942
1,1,2024,6,1,8,41.397978,2.180107,16.0,46,5,1,1.518,0.394928,0.346014,0.311594,0.271739
2,1,2024,6,1,13,41.397978,2.180107,16.0,46,5,1,1.518,0.721014,0.697464,0.650362,0.538043
3,1,2024,6,1,18,41.397978,2.180107,16.0,46,5,1,1.518,0.807971,0.791667,0.800725,0.789855
4,1,2024,6,1,23,41.397978,2.180107,16.0,46,5,1,1.518,0.793478,0.817029,0.871377,0.860507


In [20]:
predictions = model.predict(submission_data)

### 8.4 Create Submission File

In [22]:
# Create final submission dataframe
final_submission = pd.DataFrame({'percentage_docks_available': predictions})

# Adding index column to final submission, starting at 0
final_submission['index'] = range(len(submission_data))

# Reorder columns to put index first
final_submission = final_submission[['index', 'percentage_docks_available']]
final_submission.to_csv("./submission_data.csv", mode='a', index=False, header=True)
